In [1]:
import alignment, speech, applause_list, sentence, file_loader, text_features,re
import json, os, pickle, numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from IPython.display import Audio
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import matplotlib.pyplot as plt
from tqdm import tqdm
import librosa
%matplotlib inline

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution2D, Conv1D, MaxPooling2D, MaxPooling1D, Flatten, Dropout
import keras.optimizers
from keras.models import load_model
import keras.regularizers
from keras.regularizers import l2, l1

from keras.layers import LSTM
from keras.layers import regularizers
from keras.layers import Bidirectional

Using TensorFlow backend.


In [3]:
def get_preds_and_sents(s,start_index, end_index):
    sents = [s.alignment.get_preceding_chars(i) for i in range(start_index,end_index)]
    p = preds[start_index:end_index]
    return zip(sents, p)

In [112]:
all_files = file_loader.get_non_duplicate_file_endings()

In [ ]:
reload(speech)
reload(alignment)

<module 'alignment' from 'alignment.pyc'>

In [ ]:
speeches = []
for f in tqdm(all_files):
    speeches.append(speech.Speech(f))

 35%|███▌      | 124/354 [00:54<01:03,  3.62it/s]

In [136]:
speeches = [s for s in speeches if s.alignment.alignments is not None]

In [100]:
train_speeches, test_speeches = train_test_split(speeches,test_size=0.2, random_state=43)

In [121]:
s = speeches[0]

In [110]:
reload(speech)
reload(alignment)
s=speech.Speech('bernie_sanders_14')
words = s.alignment.get_clean_words()

In [92]:
len(s.alignment.get_phrase_text_and_times(s.alignment.clean_words))

537

In [134]:
s.load_librosa()

In [160]:
indices = s.alignment.get_indices_from_char_span(start_char,end_char)
start, end = s.alignment.get_start_end_times_from_indices(indices)
print start, end

next_start_char, next_end_char = s.alignment.sentence_list[i+1].start_index,s.alignment.sentence_list[i+1].end_index
next_indices = s.alignment.get_indices_from_char_span(next_start_char,next_end_char)
next_start, next_end = s.alignment.get_start_end_times_from_indices(next_indices)
print next_start, next_end

138.63 152.5
152.599999 163.53


In [161]:
# Sentence Audio
Audio(s.y[int((start)*s.sr):int((end)*s.sr)],rate=s.sr)

In [162]:
# Between Audio
if next_start > end:
    Audio(s.y[int((end)*s.sr):int((next_start)*s.sr)],rate=s.sr)

In [9]:
s.load_librosa()

In [10]:
s.load_librosa_separated()

In [40]:
i = 18
end_of_previous_app = instances[i-1][1]
start_of_this_app = instances[i][0]
end_of_this_app = instances[i][1]
print instances[i][1] - instances[i][0]
(end_of_previous_app,start_of_this_app)

7.77933


(338.99277, 350.48759)

In [42]:
Audio(s.y[int(end_of_previous_app*s.sr):int((start_of_this_app+1.5)*s.sr)],rate=s.sr)

In [43]:
Audio(s.y[int(start_of_this_app*s.sr):int(end_of_this_app*s.sr)],rate=s.sr)

In [28]:
s.alignment.alignment_file

'/data/corpora/cspan/alignments/mike_pence/mike_pence_4/mike_pence_4.json'

In [136]:
X = []
y = []
for s in tqdm(train_speeches):
    preds = s.applause_preds_by_second
    for i in range(5,len(preds)):
        raw_text = s.alignment.get_preceding_chars(i)
        if raw_text != '':
            tf = text_features.TextFeatures(raw_text)
            vec = tf.get_mean_vector(tf.sentences)
            lag = preds[i-5:i]
            vec = np.hstack([vec,lag])
            X.append(vec)
            y.append(preds[i])
X_train = X; y_train = y

100%|██████████| 283/283 [39:49<00:00,  7.03s/it]
